In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import altair as alt
from sklearn.model_selection import train_test_split

df_dtypes = {
    'category': 'str',
    'title': 'str',
    'body': 'str',
    'amenities': 'str',
    'currency': 'str',
    'fee': 'str',
    'has_photo': 'str',
    'pets_allowed': 'str',
    'price_display': 'str',
    'price_type': 'str',
    'address': 'str',
    'cityname': 'str',
    'state': 'str',
    'source': 'str'
}
df = pd.read_csv("./properties_us.csv", sep=';', encoding='ISO-8859-1', dtype=df_dtypes)
train_set, test_set = train_test_split(df, test_size=.2, random_state=42)
print(df.dtypes)

id                 int64
category          object
title             object
body              object
amenities         object
bathrooms        float64
bedrooms         float64
currency          object
fee               object
has_photo         object
pets_allowed      object
price            float64
price_display     object
price_type        object
square_feet        int64
address           object
cityname          object
state             object
latitude         float64
longitude        float64
source            object
time               int64
dtype: object


Lot of redundant features. 'body' contains text-description of property, 'currency' is predominately USD, 'fee' is >99% no and i don't even know what it means, 'source' is whatever web-platform the data was scraped from. Additionally i'll want to convert whatever string features into numerical ones.

In [39]:
df_clean = df.drop(['body', 'currency', 'fee', 'price_display', 'source'], axis=1)

def PetCount (x):
    i = 0
    if isinstance(x, str):
        i+=1+ x.count(',')
    return i

# Has photo Yes=1, No=0
df_clean['has_photo'] = (df_clean['has_photo'] != 'No').astype(int)
# Count num of pettypes allowed
df_clean['pets_allowed'] = df_clean['pets_allowed'].replace({pd.NaT: 0, None: 0})
df_clean['pets_allowed'] = df_clean['pets_allowed'].apply(lambda x: PetCount(x))
# Don't like imperial units
df_clean['square_meters'] = (df_clean['square_feet'] * 0.09290304).round(0)
df_clean = df_clean.drop('square_feet', axis=1)
# Drop the 4 records that are not monthly payment
df_clean = df_clean[df_clean['price_type'] == 'Monthly']
df_clean = df_clean.drop('price_type', axis=1)
df_clean['amenities'] = df_clean['amenities'].replace({np.nan: None})
